In [7]:
from transformers import pipeline

# Load the fine-tuned model
model = pipeline('text2text-generation', model='../data/MathBert_test', tokenizer='tbs17/MathBERT-custom')

# Provide a math problem to the model
problem = "What is the sum of 2 and 3?"
result = model(problem, max_length=32, do_sample=False)

# Extract the answer from the generated text
answer = result[0]['generated_text'].strip()
print(answer)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
The model 'BertLMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


what is the sum of 2 and 3?


In [1]:
from transformers import BartModel, BartTokenizer

# Load the fine-tuned model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartModel.from_pretrained('facebook/bart-large')

# Provide a math problem to the model
problem = "What is the sum of 2 and 3?"
inputs = tokenizer(problem, return_tensors='pt')

# Generate the output
outputs = model(**inputs)

# Decode the output
last_hidden_states = outputs.last_hidden_state

# Convert tensor to list of token ids
token_ids = last_hidden_states[0].tolist()

# Decode the token ids to get the output text
decoded_output = tokenizer.decode(token_ids, skip_special_tokens=True)
print(decoded_output)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

# Load MathBert pretrained model

Use MathBert from Jia Tracy Shen, Michiharu Yamashita, Ethan Prihar, Neil Heffernan, Xintao Wu, Ben Graff, Dongwon Lee (2023) "MathBERT: A Pre-trained Language Model for General NLP Tasks in Mathematics Education". . doi:[10.48550/arXiv.2106.07340](https://doi.org/10.48550/arXiv.2106.07340)

In [ ]:
# Create new index
train_idx = [i for i in range(len(train.index))]
test_idx = [i for i in range(len(test.index))]
val_idx = [i for i in range(len(val.index))]

# Convert to numpy
x_train = train['text'].values[train_idx]
x_test = test['text'].values[test_idx]
x_val = val['text'].values[val_idx]

y_train = train['label'].values[train_idx]
y_test = test['label'].values[test_idx]
y_val = val['label'].values[val_idx]


In [ ]:
train = Dataset.from_pandas(train, preserve_index=False)
val = Dataset.from_pandas(val, preserve_index=False)
test = Dataset.from_pandas(test, preserve_index=False)

In [ ]:
# # Pytorch Version is available via Hugging Face Transformers
# from transformers import AutoTokenizer
# from transformers import AutoModelForMaskedLM

# TEST_MODEL = r'tbs17/MathBERT-custom'

# # Download the MathBERT -basevocab
# # tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT")
# # model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT")

# # Download the MathBERT -mathvocab
# tokenizer = AutoTokenizer.from_pretrained("tbs17/MathBERT-custom")
# model = AutoModelForMaskedLM.from_pretrained("tbs17/MathBERT-custom")

Some weights of the model checkpoint at tbs17/MathBERT-custom were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def tokenization(items):
    tokenized = tokenizer(items['text'], max_length = 256, padding = 'max_length', truncation=True)
    return tokenized

def tokenization_labels(items):
    tokenized = tokenizer(items['text'], max_length = 256, padding = 'max_length', truncation=True)
    tokenized['label'] = tokenizer(items['label'], max_length = 256, padding = 'max_length', truncation=True)['input_ids']
    return tokenized

In [ ]:
train = train.map(tokenization, batched=True)

Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

In [ ]:
val = val.map(tokenization, batched=True)

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

In [ ]:
test = test.map(tokenization_labels, batched=True)

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline, TrainingArguments, Trainer, AutoModelForQuestionAnswering
from trl import SFTTrainer
import evaluate
import numpy as np

In [ ]:
metric = evaluate.load('f1')

def compute_metrics(pred):
    output, labels = pred
    return metric.compute(predictions=np.argmax(output, axis=-1), references=labels)

args = TrainingArguments(
    output_dir='/Code/data/MathQA_DB/temp',
    evaluation_strategy="epoch", # Reports the metric after each epoch
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
)

trainer = SFTTrainer(
    model = TEST_MODEL,
    args = args,
    train_dataset = train,
    eval_dataset = val,
    tokenizer=tokenizer,
    max_seq_length=256,
    dataset_text_field='text',
)

d:\Anaconda3\envs\pytorch\lib\site-packages\trl\trainer\sft_trainer.py:125: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


Map:   0%|          | 0/14656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

In [ ]:
import torch
torch.cuda.is_available = lambda : False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
trainer.train()

  0%|          | 0/5496 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.8319, 'learning_rate': 4.5451237263464344e-05, 'epoch': 0.27}
{'loss': 0.0425, 'learning_rate': 4.090247452692867e-05, 'epoch': 0.55}
{'loss': 0.0193, 'learning_rate': 3.6353711790393013e-05, 'epoch': 0.82}


  0%|          | 0/229 [00:00<?, ?it/s]

{'eval_loss': 0.006190407555550337, 'eval_runtime': 12.514, 'eval_samples_per_second': 146.396, 'eval_steps_per_second': 18.299, 'epoch': 1.0}
{'loss': 0.0107, 'learning_rate': 3.1804949053857355e-05, 'epoch': 1.09}
{'loss': 0.0049, 'learning_rate': 2.725618631732169e-05, 'epoch': 1.36}
{'loss': 0.0038, 'learning_rate': 2.2707423580786028e-05, 'epoch': 1.64}
{'loss': 0.0035, 'learning_rate': 1.8158660844250363e-05, 'epoch': 1.91}


  0%|          | 0/229 [00:00<?, ?it/s]

{'eval_loss': 0.0029049436561763287, 'eval_runtime': 13.8125, 'eval_samples_per_second': 132.634, 'eval_steps_per_second': 16.579, 'epoch': 2.0}
{'loss': 0.0022, 'learning_rate': 1.3609898107714703e-05, 'epoch': 2.18}
{'loss': 0.0014, 'learning_rate': 9.06113537117904e-06, 'epoch': 2.46}
{'loss': 0.0008, 'learning_rate': 4.512372634643377e-06, 'epoch': 2.73}


  0%|          | 0/229 [00:00<?, ?it/s]

{'eval_loss': 0.002342228777706623, 'eval_runtime': 12.5223, 'eval_samples_per_second': 146.299, 'eval_steps_per_second': 18.287, 'epoch': 3.0}
{'train_runtime': 1071.9564, 'train_samples_per_second': 41.017, 'train_steps_per_second': 5.127, 'train_loss': 0.0838296020964738, 'epoch': 3.0}


TrainOutput(global_step=5496, training_loss=0.0838296020964738, metrics={'train_runtime': 1071.9564, 'train_samples_per_second': 41.017, 'train_steps_per_second': 5.127, 'train_loss': 0.0838296020964738, 'epoch': 3.0})

In [ ]:
trainer.save_model('/Code/data/MathBert_test')

In [ ]:
from transformers import pipeline
import re

# Load the fine-tuned model
model = pipeline('text2text-generation', model='/Code/data/MathBert_test', tokenizer='tbs17/MathBERT-custom')

# Provide a math problem to the model
problem = "What is the sum of 2 and 3?"
result = model(problem, max_length=32, do_sample=False)

# Extract the formula from the generated text
formula = result[0]['generated_text'].strip()

# Check if the formula is valid
if re.match(r'^\d+(\s*[+\-*/]\s*\d+)*$', formula):
    print(f'The model converted the problem "{problem}" to the formula "{formula}"')
else:
    print(f'The model failed to convert the problem "{problem}" to a valid formula')


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
The model 'BertLMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


The model failed to convert the problem "What is the sum of 2 and 3?" to a valid formula


## Test fine-tuned model

THe kernel kept crashing; Use Mathsage_test instead.

In [ ]:
from transformers import pipeline

# Load the fine-tuned model
model = pipeline('text2text-generation', model='/Code/data/MathBert_test', tokenizer='tbs17/MathBERT-custom')

# Provide a math problem to the model
problem = "What is the sum of 2 and 3?"
result = model(problem, max_length=32, do_sample=False)

# Extract the answer from the generated text
answer = result
print(answer)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


: 